# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [1]:
import requests
import time
import re
from functools import wraps

from random import randint

In [2]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [3]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'Время выполнения функции {func.__name__}: {end - start:.16f} секунд\n')
        return result

    return wrapper

## Задание 2

In [4]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """

    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f'Функция {func.__name__} вызвана с параметрами:')
        print(f'{args}, {kwargs}\n')
        return func(*args, **kwargs)

    return wrapper

## Задание 3

In [5]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """

    cnt = 0

    @wraps(func)
    def wrapper(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        result = func(*args, **kwargs)
        print(f"Функция была вызвана: {cnt} {'раз' if cnt%10 not in (2, 3, 4) else 'раза'}\n")
        return result

    return wrapper

## Задание 4

In [6]:
def memo(func):
    """
    Декоратор, запоминающий результаты исполнения функции func, чьи аргументы args должны быть хешируемыми
    """
    cache = {}

    @wraps(func)
    def fmemo(*args):
        # nonlocal cache
        hash_args = hash(args)
        if hash_args not in cache:
            cache[hash_args] = func(*args)

        return cache[hash_args]

    fmemo.cache = cache
    return fmemo

## Тестирование

In [7]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

Функция word_count вызвана с параметрами:
('whole',), {}

Время выполнения функции word_count: 0.5747511020000005 секунд

Функция была вызвана: 1 раз

Cлово whole встречается 176 раз


In [8]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [9]:
# измеряем время выполнения
%%timeit
fib(24)

19.3 ms ± 1.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
@memo
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [11]:
# измеряем время выполнения
%%timeit
fib(24)

297 ns ± 8.61 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


**Ускорение на два порядка.**

### Дополнительный декоратор **сохраняющий** в глобальную переменную **cnt** количество вызовов декорируемой функции (без print).

In [12]:
cnt = 0

def counter2var(func):
    """
    Декоратор, считающий и сохраняющий количество вызовов декорируемой функции в глобальную переменную cnt
    """

    @wraps(func)
    def wrapper(*args, **kwargs):
        global cnt
        cnt += 1
        result = func(*args, **kwargs)
        return result

    return wrapper

In [13]:
cnt = 0

@counter2var
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [14]:
fib(32)
print('Количество вызовов функции:', cnt)

Количество вызовов функции: 7049155


In [15]:
cnt = 0

@memo
@counter2var
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [16]:
fib(32)
print('Количество вызовов функции:', cnt)

Количество вызовов функции: 33


Оптимизация работы функции, за счет кэширования выходных значений, очень заметна при рекурсивной функции.